In [96]:
import time
import csv
import pandas as pd
import requests
import json
from typing import List
from itertools import chain

In [3]:
# %pip install polars
import polars as pl

In [65]:
from extract_data import get_URLs,\
get_incident_ids, generate_canadian_avalanche_data

In [92]:
def multi_get_incident_ids(url):
    """Given a list of URLs, return the incident ids found at each URL.

    params: URLS: List[str]
    returns: ids: List[str]
    """
    ids = pd.read_json(url)['results'].apply(lambda row: row['id'])
    return ids.to_list()

In [70]:
url = "https://incidents.avalanche.ca/public/incidents/?format=json"

In [71]:
URLS = get_URLs(url)

In [72]:
URLS

['https://incidents.avalanche.ca/public/incidents/?format=json',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=2',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=3',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=4',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=5',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=6',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=7',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=8',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=9',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=10',
 'https://incidents.avalanche.ca/public/incidents/?format=json&page=11']

In [91]:
ids = get_incident_ids(URLS)

In [74]:
from multiprocessing import Pool



In [93]:
with Pool() as pool:
    results = pool.map(multi_get_incident_ids, URLS)
    final = list(chain.from_iterable(results))
    # Process results as needed


In [94]:
len(final)

504

In [14]:
len(ids)

504

In [101]:
def generate_canadian_avalanche_data(incident_ids: List[str], file) -> str:
    df = pd.DataFrame()
    for id in incident_ids:
        data = requests.get(
            "https://incidents.avalanche.ca/public/incidents/{}/?format=json".format(id)
        ).json()
        new_df = pd.json_normalize(data)[["ob_date", "location_coords", "location_coords_type"]]
        df = pd.concat([df, new_df])
    df.to_csv(file, index=False)

    return "Data was saved."

In [100]:
import requests
import pandas as pd
from multiprocessing import Pool
import csv

def multi_generate_canadian_avalanche_data(args):
    incident_id, file = args
    data = requests.get("https://incidents.avalanche.ca/public/incidents/{}/?format=json".format(incident_id)).json()
    df = pd.json_normalize(data)[["ob_date", "location_coords", "location_coords_type"]]
    
    # Open the CSV file for writing in append mode
    with open(file, 'a', newline='\n') as csv_file:
        # Create a CSV writer
        csv_writer = csv.writer(csv_file)

        # Write the data to the CSV file
        csv_writer.writerow(df.values.flatten())

    return 

# Define the CSV headers
csv_headers = ["ob_date", "location_coords", "location_coords_type"]

# Open the CSV file for writing
with open('./multiout.csv', 'w', newline='\n') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the headers to the CSV file
    csv_writer.writerow(csv_headers)

    # Create a list of tuples, each containing arguments for a single function call
    args_list = [(incident_id, csv_file.name) for incident_id in ids[:50]]

    with Pool() as p:
        results = p.map(multi_generate_canadian_avalanche_data, args_list)

    # Process the results if needed
    print(results)
q

['Data from 761a8854-758d-4a16-8d90-75d1b999ef02 was saved.', 'Data from 6dad9a7f-e732-43cf-8058-5fcac24c7041 was saved.', 'Data from a3968149-3ee5-4360-8efb-ed8bc3f331c4 was saved.', 'Data from 913e84e6-4785-409e-859e-f6456b2fb27c was saved.', 'Data from 645d13aa-4a96-4f11-a787-ded7a0051d63 was saved.', 'Data from a764c492-bb8d-401d-b7b4-ec5d7915b98e was saved.', 'Data from 1222afb5-f434-48b9-a92b-19d472c3f914 was saved.', 'Data from e8005ae1-313a-4a34-a327-16e6dadb19d6 was saved.', 'Data from bd1d03e6-ed8e-4101-93e4-3c214d3d39ca was saved.', 'Data from e120382f-80cf-4aaf-8a13-7f028e88879d was saved.', 'Data from 0be6f146-74fc-45fb-a748-799f0599aa23 was saved.', 'Data from 88fb4749-8b10-4640-8aa0-acdffeebfe76 was saved.', 'Data from bb4569b4-a158-4ca7-bb4c-6cfc58b6498d was saved.', 'Data from ddbe24b9-40bc-40ba-b8b5-94dd4e02e9dd was saved.', 'Data from ce706316-3687-45f0-a8db-fb63b7000ea5 was saved.', 'Data from 8bc4720d-498c-4793-81ef-c43db9f36ca4 was saved.', 'Data from 6a3a4698-d04

In [104]:

with open('./multiout.csv', 'w', newline='\n') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the headers to the CSV file
    csv_writer.writerow(csv_headers)

    # Create a list of tuples, each containing arguments for a single function call
    args_list = [(incident_id, csv_file.name) for incident_id in ids]

    with Pool() as p:
        results = p.map(multi_generate_canadian_avalanche_data, args_list)

    # Process the results if needed
    print(results)

['Data from 761a8854-758d-4a16-8d90-75d1b999ef02 was saved.', 'Data from 6dad9a7f-e732-43cf-8058-5fcac24c7041 was saved.', 'Data from a3968149-3ee5-4360-8efb-ed8bc3f331c4 was saved.', 'Data from 913e84e6-4785-409e-859e-f6456b2fb27c was saved.', 'Data from 645d13aa-4a96-4f11-a787-ded7a0051d63 was saved.', 'Data from a764c492-bb8d-401d-b7b4-ec5d7915b98e was saved.', 'Data from 1222afb5-f434-48b9-a92b-19d472c3f914 was saved.', 'Data from e8005ae1-313a-4a34-a327-16e6dadb19d6 was saved.', 'Data from bd1d03e6-ed8e-4101-93e4-3c214d3d39ca was saved.', 'Data from e120382f-80cf-4aaf-8a13-7f028e88879d was saved.', 'Data from 0be6f146-74fc-45fb-a748-799f0599aa23 was saved.', 'Data from 88fb4749-8b10-4640-8aa0-acdffeebfe76 was saved.', 'Data from bb4569b4-a158-4ca7-bb4c-6cfc58b6498d was saved.', 'Data from ddbe24b9-40bc-40ba-b8b5-94dd4e02e9dd was saved.', 'Data from ce706316-3687-45f0-a8db-fb63b7000ea5 was saved.', 'Data from 8bc4720d-498c-4793-81ef-c43db9f36ca4 was saved.', 'Data from 6a3a4698-d04

In [105]:
generate_canadian_avalanche_data(ids, './single_out.csv')

'Data was saved.'

In [48]:
%%timeit 
fetch_and_write_to_csv(ids[:50], output_csv_file)

1.96 s ± 60.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%%timeit
generate_canadian_avalanche_data(ids[:50])

2.09 s ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
ids[0]

'761a8854-758d-4a16-8d90-75d1b999ef02'

In [51]:

src = "https://incidents.avalanche.ca/public/incidents/6dad9a7f-e732-43cf-8058-5fcac24c7041/?format=json"
pd.read_json(src)

ValueError: All arrays must be of the same length

In [52]:
data = requests.get(src).json()
pd.json_normalize(data)

,id,ob_date,location,location_desc,location_coords,location_coords_type,location_elevation,location_province,num_involved,num_injured,...,weather_obs.temp_min,weather_obs.temp_trend,weather_obs.wind_speed,weather_obs.wind_dir,weather_obs.sky,weather_obs.precip,snowpack_obs.hs,snowpack_obs.hn24,snowpack_obs.hst,snowpack_obs.hst_reset
0,6dad9a7f-e732-43cf-8058-5fcac24c7041,2023-04-22,Lake Louise West Bowl,Lake Louise Ski Resort,"[51.467515, -116.145756]",Lat/lng,2550.0,AB,2,1,...,None,None,None,None,None,None,None,None,None,None


In [56]:
pd.read_json(url)["results"]

0     {'id': '761a8854-758d-4a16-8d90-75d1b999ef02',...
1     {'id': '6dad9a7f-e732-43cf-8058-5fcac24c7041',...
2     {'id': 'a3968149-3ee5-4360-8efb-ed8bc3f331c4',...
3     {'id': '913e84e6-4785-409e-859e-f6456b2fb27c',...
4     {'id': '645d13aa-4a96-4f11-a787-ded7a0051d63',...
5     {'id': 'a764c492-bb8d-401d-b7b4-ec5d7915b98e',...
6     {'id': '1222afb5-f434-48b9-a92b-19d472c3f914',...
7     {'id': 'e8005ae1-313a-4a34-a327-16e6dadb19d6',...
8     {'id': 'bd1d03e6-ed8e-4101-93e4-3c214d3d39ca',...
9     {'id': 'e120382f-80cf-4aaf-8a13-7f028e88879d',...
10    {'id': '0be6f146-74fc-45fb-a748-799f0599aa23',...
11    {'id': '88fb4749-8b10-4640-8aa0-acdffeebfe76',...
12    {'id': 'bb4569b4-a158-4ca7-bb4c-6cfc58b6498d',...
13    {'id': 'ddbe24b9-40bc-40ba-b8b5-94dd4e02e9dd',...
14    {'id': 'ce706316-3687-45f0-a8db-fb63b7000ea5',...
15    {'id': '8bc4720d-498c-4793-81ef-c43db9f36ca4',...
16    {'id': '6a3a4698-d047-4082-bdea-92f4db7e63bf',...
17    {'id': 'ba14a125-29f7-4432-97ad-73a53207a5

In [58]:
# data = requests.get(url).text
pl.read_json("data.json")["results"][0]

""
struct[8]
"{""761a8854-758d-4a16-8d90-75d1b999ef02"",""2023-11-11"",""Ranger Creek"",""AB"",""Ice Climbing"",2,0,1}"
"{""6dad9a7f-e732-43cf-8058-5fcac24c7041"",""2023-04-22"",""Lake Louise West Bowl"",""AB"",""Lift Skiing Closed"",2,1,1}"
"{""a3968149-3ee5-4360-8efb-ed8bc3f331c4"",""2023-04-15"",""Thunderwater Lake"",""BC"",""Snowmobiling"",2,0,1}"
"{""913e84e6-4785-409e-859e-f6456b2fb27c"",""2023-04-11"",""Treaty Creek"",""BC"",""Mechanized Skiing"",5,1,1}"
"{""645d13aa-4a96-4f11-a787-ded7a0051d63"",""2023-03-01"",""Coppercrown Mountain"",""BC"",""Mechanized Skiing"",10,4,3}"
"{""a764c492-bb8d-401d-b7b4-ec5d7915b98e"",""2023-02-16"",""Terminator 2.5"",""BC"",""Out-of-Bounds Skiing/Snowboarding"",6,1,2}"
"{""1222afb5-f434-48b9-a92b-19d472c3f914"",""2023-02-11"",""Potato Peak"",""BC"",""Backcountry Skiing"",2,0,2}"
"{""e8005ae1-313a-4a34-a327-16e6dadb19d6"",""2023-01-23"",""Akolkolex"",""BC"",""Mechanized Skiing"",3,1,2}"
"{""bd1d03e6-ed8e-4101-93e4-3c214d3d39ca"",""2023-01-21"",""Oasis"",""BC"",""Snowmobiling"",2,0,1}"


In [25]:
data.text

'{"count":504,"next":"http://incidents.avalanche.ca/public/incidents/?format=json&page=2","previous":null,"results":[{"id":"761a8854-758d-4a16-8d90-75d1b999ef02","date":"2023-11-11","location":"Ranger Creek","location_province":"AB","group_activity":"Ice Climbing","num_involved":2,"num_injured":0,"num_fatal":1},{"id":"6dad9a7f-e732-43cf-8058-5fcac24c7041","date":"2023-04-22","location":"Lake Louise West Bowl","location_province":"AB","group_activity":"Lift Skiing Closed","num_involved":2,"num_injured":1,"num_fatal":1},{"id":"a3968149-3ee5-4360-8efb-ed8bc3f331c4","date":"2023-04-15","location":"Thunderwater Lake","location_province":"BC","group_activity":"Snowmobiling","num_involved":2,"num_injured":0,"num_fatal":1},{"id":"913e84e6-4785-409e-859e-f6456b2fb27c","date":"2023-04-11","location":"Treaty Creek","location_province":"BC","group_activity":"Mechanized Skiing","num_involved":5,"num_injured":1,"num_fatal":1},{"id":"645d13aa-4a96-4f11-a787-ded7a0051d63","date":"2023-03-01","location